In [1]:
from langchain_community.document_loaders import YoutubeLoader
# from spire.doc import *
# from spire.doc.common import *
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_community.llms import HuggingFaceEndpoint
import re
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [3]:
youtube_url=input("Youtube Link:")



In [4]:
loader = YoutubeLoader.from_youtube_url(
    youtube_url, add_video_info=False
)

In [5]:
documents = loader.load()  # Assuming this returns a list of Document objects


In [6]:
documents

[Document(page_content="- I've spent most of my career trying to get into a chip fab. Samsung's a hard no, Global Foundries was open to it in person, then ghosted me when I followed up and Intel has said no to me, every single time I've asked except one. Honestly, I'm pretty overwhelmed today because I'm gonna be\ntaking three major items off my bucket list. I'm visiting Israel for the first time, I'm gonna be going deep\ninto the heart of Intel's state of the art fab 28 and I get to tell you about our sponsor. - Zoho CRM, Zoho CRM is a 360 degree solution that offers an intuitive UI, AI predictions and a design studio to help you get your sales done faster. Get 50% off with the code\nZCRM50 using the link below. (upbeat music) (upbeat music) - My tour guide today\ncouldn't be more experienced. Dani is the Co-General Manager of Worldwide Semiconductor Manufacturing and started at Intel working\nin fab 8 in Jerusalem back when state of the\nart was the Pentium one. Microprocessor fabric

In [7]:
def clean_text(text):
    """
    Cleans the extracted text by removing unnecessary characters and whitespace.

    Args:
    text (str): Extracted text.

    Returns:
    str: Cleaned text.
    """
    # Remove extra whitespace and newlines
    cleaned_text = re.sub(r'\s+', ' ', text)
    # Remove non-printable characters
    cleaned_text = re.sub(r'[^\x00-\x7F]+', '', cleaned_text)
    # Remove any leading or trailing whitespace
    cleaned_text = cleaned_text.strip()
    return cleaned_text

In [8]:
script=clean_text(documents[0].page_content)

In [9]:
(script)

"- I've spent most of my career trying to get into a chip fab. Samsung's a hard no, Global Foundries was open to it in person, then ghosted me when I followed up and Intel has said no to me, every single time I've asked except one. Honestly, I'm pretty overwhelmed today because I'm gonna be taking three major items off my bucket list. I'm visiting Israel for the first time, I'm gonna be going deep into the heart of Intel's state of the art fab 28 and I get to tell you about our sponsor. - Zoho CRM, Zoho CRM is a 360 degree solution that offers an intuitive UI, AI predictions and a design studio to help you get your sales done faster. Get 50% off with the code ZCRM50 using the link below. (upbeat music) (upbeat music) - My tour guide today couldn't be more experienced. Dani is the Co-General Manager of Worldwide Semiconductor Manufacturing and started at Intel working in fab 8 in Jerusalem back when state of the art was the Pentium one. Microprocessor fabrication experts can safely skip

In [10]:

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"  
llm = HuggingFaceEndpoint(
    repo_id=repo_id, temperature= 0.75, model_kwargs= {'max_length': 8192}, max_new_tokens=4096)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/shreya/.cache/huggingface/token
Login successful


In [22]:
prompt="""As an expert summarizer, your job is to condense the following YouTube video transcript into a summary of 900-1000 words. Focus on capturing the essential points and main ideas, leaving out any unnecessary details.

Example of a good summary:
---
Transcript: 
"In this video, we delve into the impact of climate change on global weather patterns. The speaker explains how rising temperatures are causing more frequent and severe weather events, including hurricanes, floods, and droughts. Scientific data is presented to show the correlation between human activities and climate change. The video also discusses possible solutions to mitigate these effects, such as reducing carbon emissions and adopting sustainable practices. Interviews with climate scientists provide further insights into the urgency of addressing climate change."

Summary:
"This video examines the effects of climate change on global weather patterns, highlighting the link between rising temperatures and increased frequency of extreme weather events. It presents scientific data demonstrating the human influence on climate change and discusses mitigation strategies like reducing carbon emissions and adopting sustainable practices. Insights from climate scientists underscore the urgency of taking action against climate change."
---

Here is the transcript you need to summarize:
"""



In [23]:
prompt_template = PromptTemplate(
        input_variables=["text"],
        template=prompt+"""
{text}
"""
    )

    # Create an LLMChain with the LLM and prompt
chain = prompt_template | llm
    
    # Run the chain to summarize the text
summary = chain.invoke(script)

In [24]:
print(summary)


Summary:

In this YouTube video, the host takes viewers on a tour of Intel's Fab 28 in Israel, providing an inside look at the chip manufacturing process. Intel's state-of-the-art fab covers an area of four football fields and operates on four shifts a day, 24/7, except for Yom Kippur. The tour begins at the beginning of the process, with a sliced silicon ingot, which is then transformed into a CPU die through various processes such as implantation, diffusion, lithography, etching, deposition, and planarization. The wafer goes through these processes multiple times before it's ready for packaging and quality control.

The host also discusses the importance of consistency and the high cost of these machines. Intel has a multi-layered structure in the fab, with each layer containing different machinery for specific processes. The tour includes a demonstration of Intel's use of augmented reality for maintenance and training purposes. The video concludes with a look at the remote operatio

In [25]:

Summary:

In this YouTube video, the host takes viewers on a tour of Intel's Fab 28 in Israel, providing an inside look at the chip manufacturing process. Intel's state-of-the-art fab covers an area of four football fields and operates on four shifts a day, 24/7, except for Yom Kippur. The tour begins at the beginning of the process, with a sliced silicon ingot, which is then transformed into a CPU die through various processes such as implantation, diffusion, lithography, etching, deposition, and planarization. The wafer goes through these processes multiple times before it's ready for packaging and quality control.

The host also discusses the importance of consistency and the high cost of these machines. Intel has a multi-layered structure in the fab, with each layer containing different machinery for specific processes. The tour includes a demonstration of Intel's use of augmented reality for maintenance and training purposes. The video concludes with a look at the remote operational center, which optimizes the overall throughput of the fab and ensures that as many chips as possible are produced. Intel is currently building out fab capacity with fab 38, which will be integrated with the existing fab and will manufacture products for third parties. The video offers a fascinating glimpse into the complex and costly world of chip manufacturing.


SyntaxError: unterminated string literal (detected at line 3) (1551287128.py, line 3)

In [26]:
len(summary)

1368

In [27]:
repo_id2 = "Telugu-LLM-Labs/Indic-gemma-7b-finetuned-sft-Navarasa-2.0"  
llm2 = HuggingFaceEndpoint(
    repo_id=repo_id, temperature= 0.75, model_kwargs= {'max_length': 8192}, max_new_tokens=4096)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/shreya/.cache/huggingface/token
Login successful


In [28]:
language_needed=input("What language do you want translation in?")

In [29]:
translation_prompt = """Translate the given text into {language_needed}". Understand the text properly and correct any grammatical mistakes. Output only the translated text in the form of points.:\n{text}"""

In [30]:
tr_input={
    'text':summary,
    'language_needed':language_needed
}

In [31]:
prompt_template = PromptTemplate(
        input_variables=["text"],
        template=translation_prompt
    )

    # Create an LLMChain with the LLM and prompt
chain = prompt_template | llm
    
    # Run the chain to summarize the text
translated_summary = chain.invoke(tr_input)

In [32]:
print(translated_summary)



हिंदी में ट्रांसलेट:

यह YouTube वीडियो में वार्ता साथ हैं लोगों को Intel ऑफ 28 इस्राएल में चिप बनाने का जगह दिखाते हैं, जिसमें चिप बनाने से अंतर्रित दरशन प्रदेशित किया जाता है। Intel का स्थानीय लाभ के साथे चौथाली फूटबॉल खेत का कार्य होता है और चौकीदारों के चार शिफ्टों से रात्रियाँ तक रूपरेखा और 24/7 के बिना योम किप्पुर के लिए। प्रकर्ष का प्रकरार से शुरुआत होता है सिलिकन इंगौट से काटा हुआ, जिससे केवल एक प्रोसेसों के द्वारा कुछ प्रकार की जीवित मात्रा के CPU दायाँ बनती जाती है।

वार्ता के अनुसार यह प्रकारों के माध्यम से वाफर के लिए बहुत से बार पहुँचता है जबकि उसे पैकिजिंग और कालापन्नात्मक जानकारी के लिए सही होने तक पहुँचाई जाती है।

वार्ता ने यह संबंधित बातें भी बताये कि ताकत के अहमता और इस मशीनों की ऊँचाई का कुछ। Intel के लाभ में वाफर में एक साथित स्रुती है, जिसमें कुछ प्रकार की माशीन हैं जो केवल कुछ प्रकार के प्रोसेसों के लिए उपयोग की जाती हैं। वीडियो में एक दर्शन है कि Intel का अग्रेजड रेलिटी उपयोग करता है रोक्ने और साझा करने के लिए। वीडियो का सामग्री अपना देखने से पहले एक दिशा से ए

In [ ]:
Prompt 2

"\n\nPoints:\n\n1. ಈ ವಿಜಯಿತ್ತುವನ್ನು Intel fab 28, ಅದಕ್ಕಾಗಿ ಸ್ವತಂತ್ರ ಮಿಕ್ರೋಪ್ರೋಸೆಸರ್ ಸಾಧನೆ ಮುಕ್ತಾವರಧಿಕ ಮಿಕ್ರೋಪ್ರೋಸೆಪ್ರೋಜೆಟ್ ಸಾಧನೆಗಳ ಮೂಲಕ ಆಗಿರುವ ಮತ್ತು ಸದ್ಯವರ್ಧನ ಮಿಕ್ರೋಪ್ರೋಸೆಪ್ರೋಜೆಟ್ ಫಾಬ್ ಮಾಜಳಿಗೆ ಹೊರೆದು ಸಾಧನೆ ಕಳಸಿದೆ.\n2. ಮಿಕ್ರೋಪ್ರೋಸೆಪ್ರೋಜೆಟ್ ಸಾಧನೆಯಲ್ಲಿ ಬಳಕೆ ಸಿಲಿಕನ್ ಇಗೋತ್ ಆಗಿರುವ ಬಸೆಗೆ ಹೊರಟು ಹೊರಮತೆಗಳ ಮೂಲಕ ಮಾಡುವ ಬಸೆ ಸಂಶ್ಲಕ್ಷತೆಯಂತಾ ಮತ್ತು ಶಕ್ತಿಷ್ಠತೆ ಅಧಿಕರಿಸುವ ಆಗ್ರೆಷ್ಮ ಅನುವಾದ ಮಿಕ್ರೋಪ್ರೋಸೆಪ್ರೋಜೆಟ್ ಸಾಧನೆಯನ್ನು ನಮ್ಮ ಹೆಸರೆ ಹೊರಟೆವು.\n3. ಮಿಕ್ರೋಪ್ರೋಸೆಪ್ರೋಜೆಟ್ ಸಾಧನೆಯನ್ನು ನೋಡಲು ಅನುವಾದ ಮಿಕ್ರೋಪ್ರೋಸೆಪ್ರೋಜೆಟ್ ಸಂಶ್ಲಕ್ಷತೆಯಲ್ಲಿ ಸಂಖ್ಯೆ ಮತ್ತು ಪರಿಕರ್ಷಕತೆ ಅಧಿಕರಿಸುವ ಆಗ್ರೆಷ್ಮ ಮತ್ತು ಪ್ರತಿಸಂಖ್ಯೆ ಅಧಿಕರಿಸುವ ಆಶಾಸನೆಯಾದ ಆತ್ಮವಿಷಯವನ್ನು ಹೊರಗೆ ಕೇಳಿಸಲು ಹೇಳಿದೆ.\n4. ಮನೆಜಮಣಿಯಲ್ಲಿ ಒಳಗೆ ಆಗಿರುವ ಮನುಫァಕ್ಷಿಯಳ್ಲಿಗೆ ಮನೆಜಮಣಿಯಲ್ಲಿ ಹೊರಟವೆಳ್ಲಳೆ ಮನುಫァಕ್ಷಿಯನ್ನು ಹೊರಗೆ ಕೇಳಿಸಲು ಮುಕ್ತಾವರಧಿಕ ಬಾಧ್ಯತೆಯನ್ನು ಹೇಳಿದೆ. ಆಗೆ ಮನೆಜಮಣಿಯಲ್ಲಿ ಹೊರಟವೆಳ್ಲೆ ಮನೆಜಮಣಿಯನ್ನು ನೋಡಲು ಆಗಿರುವ ಮಂಜಳಃ ಮಂಜಲೆ ಮತ್ತು ಅನ್ನಭಧ್ಯಶಾಲೆ ಅದಕ್ಕೆ ಮನೆಜಮಣಿಯನ್ನು ಹೇಳಿದೆ.\n5. ಮಂಜಲೆ ಮತ್ತು ಅನ್ನಭಧ್ಯಶಾಲೆಯಲ್ಲಿ ಮನೆಜಮಣಿಯರಿಗೆ ಇಮ್ಮೇಣು ಮನುಫァಕ್ಷಿಯಗಳಿಗೆ ಹೊರಗೆ ಕೇಳಿಸಲು ಆಗಿರುವ ಅನುವಾದ ಮತ್ತು ಮಿಕ್ರೋಪ್ರೋಸೆಪ್ರೋಜೆಟ್ ಸಾಧನೆಯಲ್ಲಿ ಬಸ್ತು ಮಂಜಲೆ ಮತ್ತು ಅನ್ನಭಧ್ಯಶಾಲೆಗೆ ಹೊರಗೆ ಕೇಳಿಸಲು ಮುಕ್ತಾವರಧಿಕ ಬಾಧ್ಯತೆಯನ್ನು ಹೇಳಿದೆ.\n6. ಮನೆಜಮಣಿಯನ್ನು ನೋಡಿಸಲು ಆಗಿರುವ ಅಧಿಕಾರಿಗೆ ಅವರು Intel fab 38 ಶ್ರೇಷ್ಠವಾದ ಮಿಕ್ರೋಪ್ರೋಸೆಪ್ರೋಜೆಟ್ ಸಾಧನೆಯನ್ನು ಹೇಳಿದೆ. ಆಗೆ ಆಧನೀಯವಾದ Intel IDM 2.0 ಸ್ರಾಟೆಜಿಯ ಸ್ರತ್ಯದಿಶಿಯಲ್ಲಿ ಮತ್ತು ಅಧಿಕಾರಿಗೆ ಒಳಗೆ ಆಗಿರುವ ಅನ್ಯ ಕೋಷಿಗಳಿಗೆ ಮಿಕ್ರೋಪ್ರೋಸೆಪ್ರೋಜೆಟ್ ಸಾಧನೆಗಳನ್ನು ಮುಕ್ತಾವರಧಿಕ ಶಕ್ತಿಶಾಲೆಗೆ ತಶ್ರೀವಸಾಧನೆ ಮಾಡುವ ಆತ್ಮವಿಷಯವನ್ನು ಹೇಳಿದೆ.\n7. ಮನೆಜಮಣಿಯಲ್ಲಿ ನಮ್ಮ ಹೆಸರೆ ಹೊರಟೆವು ಮನೆಜಮಣಿಯಲ್ಲಿ ತಿರುಗಿದ ಅನ್ಯ ತಪ್ಪಿಗೆ ಹೊರಟವೆಳ್ಲಳೆ ಮನೆಜಮಣಿಯನ್ನು ಹೇಳಿದೆ. ಮನೆಜಮಣಿಯನ್ನು ಹೇಳಿದೆ ಆಗ ಮನೆಜಮಣಿಯಲ್ಲಿ ಅವರಿಗೆ ಮನೆಜಮಣಿಯನ್ನು ನೋಡಲು ಮುಕ್ತಾವರಧಿಕ ಬಾಧ್ಯತೆಯನ್ನು ಹೇಳಿದೆ.\n8. ಮನೆಜಮಣಿಯನ್ನು ಹೇಳಿದೆ ಆಗ ಮನೆಜಮಣಿಯಲ್ಲಿ ಮನೆಜಮಣಿಯನ್ನು ನೋಡಲು ಮುಕ್ತಾವರಧಿಕ ಬಾಧ್ಯತೆಯನ್ನು ಹೇಳಿದೆ. ಮನೆಜಮಣಿಯಲ್ಲಿ ಹೊರಟವೆಳ್ಲೆ ಮನೆಜಮಣಿಯನ್ನು ನೋಡಲು ಮುಕ್ತಾವರಧಿಕ ಬಾಧ್ಯತೆಯನ್ನು ಹೇಳಿದೆ.\n9. ಮನೆಜಮಣಿಯಲ್ಲಿ ಹೊರಟವೆಳ್ಲೆ ಮನೆಜಮಣಿಯನ್ನು ನೋಡಲು ಮುಕ್ತಾವರಧಿಕ ಬಾಧ್ಯತೆಯನ್ನು ಹೇಳಿದೆ. ಆಗೆ ಮನೆಜಮಣಿಯಲ್ಲಿ ಮನೆಜಮಣಿಯನ್ನು ನೋಡಲು ಮುಕ್ತಾವರಧಿಕ ಬಾಧ್ಯತೆಯನ್ನು ಹೇಳಿದೆ.\n10. ಮನೆಜಮಣಿಯಲ್ಲಿ ಮನೆಜಮಣಿಯನ್ನು ನೋಡಲು ಮುಕ್ತಾವರಧಿಕ ಬಾಧ್ಯತೆಯನ್ನು ಹೇಳಿದೆ. ಆಗೆ ಮನೆಜಮಣಿಯಲ್ಲಿ ಮನೆಜಮಣಿಯನ್ನು ನೋಡಲು ಮುಕ್ತಾವರಧಿಕ ಬಾಧ್ಯತೆಯನ್ನು ಹೇಳಿದೆ.\n\nTranslation:\n\n1. In this video, the host goes on a tour of Intel's fab 28, one of the most advanced microprocessor manufacturing facilities in the world. He explains the basics of microprocessor fabrication, from the silicon ingot to the finished product, emphasizing the importance of consistency and precision throughout the process.\n2. The host explains that the consistency and precision required in microprocessor fabrication are essential in microprocessor fabrication factories like Intel fab 28.\n3. The host discusses the importance of the various manufacturing stages, including implantation, diffusion, lithography, etching, and deposition, in producing consistent and high-quality microprocessors.\n4. He mentions that the fabrication process involves several manufacturing facilities, such as manejamani (diffusion furnace), manjale (clean room), and annabhagya shala (lithography room).\n5. He also mentions that Intel is expanding its manufacturing capacity with the construction of fab 38 and its new IDM 2.0 strategy, which includes manufacturing products for third-party companies.\n6. The host expresses awe and fascination for the technological marvels inside the fab and the importance of precision and consistency in producing high-quality microprocessors.\n7. He mentions that Intel's fab 38 is a significant investment in expanding the company's manufacturing capacity.\n8. The host expresses his admiration for the advanced technology and precision required in microprocessor fabrication.\n9. He emphasizes the importance of consistency and precision in every stage of microprocessor fabrication.\n10. He reiterates the importance of consistency and precision in producing high-quality microprocessors and marvels at the technological marvels inside the fab."

In [ ]:
Prompy 3
---

1. ಈ ವಿಜಯ್ಯತ್ತ್ಸದ ವಿದ�����oshಕ್ಕೆ, ಅವನು ಅನ್ನದೆ ಈಡೆಲ್ ಫಬ್ ನಂ.ಒಟ್ಟು (ಫಬ್ 28), ಒಳ್ಳೆ ಸಾಮಗ್ರಿಯ ಮಿಕ್ರೋಪ್ರೋಸೆಷರ್ ಜನ್ಮನೆ ಬೆಳ್ಲಿಗೆ ತೋರಿಸುತ್ತಿದೆ.
2. ಮಿಕ್ರೋಪ್ರೋಸೆಷರ್ ಜನ್ಮನೆಯಲ್ಲಿ ಬಸ್ತು ಅವನ ಹೊಸೆಗಳಿಗೆ ಹೇಳಿಸುತ್ತದೆ, ಸಿಲಿಕನ್ ಇಗಟ್ ಸಾರೆ ತಿಲ್ಲದೆ ಮತ್ತು ಮತ್ತೊಂದು ಸಂಪೂರ್ಣ ಮಾಧ್ಯಧನವನ್ನು ಮಾಡುವ ಹುಡುದ್ಗಳು ಕಲಿತ್ವ ಮತ್ತು ಸಮಸ್ತ ಪ್ರಕಾರಭಾಗಗಳಿಗೆ ಸಮಸ್ತ ಸಾಮಗ್ರಿ ಮತ್ತು ಸರಿರುಗೆ ಸಮಮಥನೆಯ ಇಮಪ್ರಟ್ಟು ಅವಸ್ಥೆಗಳಿಗೆ ಸಾಧಿಸುತ್ತದೆ.
3. ತೋರಿಸುವ ಮನುфаಕ್ಟರಿಗಳು, ಇಮ್ಪ್ಲಾಂಟಟಿಶನ್, ಡಿಫುಷನ್, ಲಿಥಓಗ್ರಫಿ, ಎಟ್ಚಿಂಗ್ ಮತ್ತು ಡೆಪೋಶಿಯಲ್ಲಿ ಕಾಣಿಸುತ್ತಿದೆ.
4. ಅವನು ಈ ಫಬ್ 28 ಫಾಬ್-ಕ್ಯೂಟನ್ ಮನುಫಕ್ಟರಿಗೆ ಬಂದ ಮತ್ತು ಹೆಜ್ಜಿನ್ ಆಶೆಯಲ್ಲಿ ಬೆಳ್ಲಿಗೆ ಹೋಗುವ ರಾಜ್ಕೊಟ್ ಮತ್ತು ಇಂಟೆಲ್ ಶ್ರೀಟಿಗೆ ಹೇಳಿಸುತ್ತಿದೆ.
5. ಅವನು ಇಂಟೆಲ್ ನಿಮ್ಮೆಣ್ಣು ಫಬ್ 38 ಮನುಫಕ್ಟರಿಗೆ ನಿಜಗಳಿಗೆ ಬ್ಯೂಡ್ ಮತ್ತು ಅನ್ನ ಇಡ್ಅಮ್ 2.ಓ ಸ್ರಟೆಜಿಃ (ಸಿಡ್ಯನ್ಮೆಂಟ್ 2.ಓ) ಸ್ರಟೆಜಿಃ ಮಣ್ಡ್ಫಾಕ್ಟ್ ಮತ್ತು ಹಜ್ಜರಿಗೆ ಹೇಳಿಸುತ್ತಿದೆ.
6. ಮನುಫಕ್ಟರಿಗೆ ತೋರಿಸುವ ಹೊಸಗಳು ಇಂಟೆಲ್ ಮತ್ತು ಒಟ್ಟಮ್ ಸಾಧಿಕೆ ಮತ್ತು ಒಟ್ಟಮ್ ಶಕ್ತಿಶಾಲೆಗಳಿಗೆ ಸಮಸ್ತ ಆಶೆಯ ಮತ್ತು ಅವಸ್ಥೆಗಳಿಗೆ ಸಮಮಥನೆಯ ಮಾಹೀತ್ವವನ್ನು ಮತ್ತು ಶಕ್ತಿಶಾಲೆಗಳಿಗೆ ಬಳಸುತ್ತಿದೆ.
7. ಮನುಫಕ್ಟರಿಗೆ ಕಳುಹೆಯೇ ಹೋಗುವ ಹೊಸಗಳು ತಮ್ಮ ಸ್ವತಂತ್ರ ಮತ್ತು ತಮ್ಮ ಶಕ್ತಿಶಾಲೆಗಳಿಗೆ ಮಾಡುವ ಬೆಳ್ಲಿಗೆ ಮಾಡುತ್ತಿದೆ.
8. ಅವನು ಇಂಟೆಲ್ ಫಬ್ 28 ಫಾಬ್-ನೇಟ್ ಮನುಫಕ್ಟರಿಗೆ ಬಂದು ತಮ್ಮ ಅವಸ್ಥೆಗಳಿಗೆ ಹೇಳಿಸುತ್ತಿದೆ. ಅವನು ತಮ್ಮ ಹುಡುದ್ಗಳೆಗಳಿಗೆ ಸಮಸ್ತ ಆಶೆಯ ಮತ್ತು ಅವಸ್ಥೆಗಳಿಗೆ ಸಮಮಥನೆಯ ಮಾಹೀತ್ವವನ್ನು ಕೇಷಿಸುತ್ತಿದೆ.
9. ಅವನು ಈ ಫಬ್ 28 ಫಾಬ್-ನೇಟ್ ಮನುಫಕ್ಟರಿಗೆ ಬಂದು ಅವನ ಹೆಜ್ಜಿನ್ ಆಶೆಯಲ್ಲಿ ಹೇಳಿಸುತ್ತಿದೆ. ಅವನು ಈ ಜಾಧೆಯಲ್ಲಿ ಜನ್ಮಮಾಡಿತ ಮಿಕ್ರೋಪ್ರೋಸೆಷರ್ಗಳಿಗೆ ಅವಸ್ಥೆಗಳಿಗೆ ಸಮಮಥನೆಯ ಶಕ್ತಿಶಾಲೆಗಳಿಗೆ ಬಳಸುತ್ತಿದೆ.
10. ಅವನು ಈ ವಿಜಯ್ಯತ್ಸದ ವಿದೇಶಕ್ಕೆ ಮತ್ತು ಈ ಫಬ್ 28 ಮನುಫಕ್ಟರಿಗೆ ಬಂದು ತಮ್ಮ ಸ್ವತಂತ್ರ ಮತ್ತು ಒಟ್ಟಮ್ ಸಾಧಿಕೆ ಮತ್ತು ಮತ್ತೊಂದು ಸಂಪೂರ್ಣ ಆಶೆಯ ಮತ್ತು ಅವಸ್ಥೆಗಳಿಗೆ ಸಮಮಥನೆಯ ಮಾಹೀತ್ವವನ್ನು ಕೇಷಿಸುತ್ತಿದೆ.
